In [4]:
import numpy as np

# 1. 定义一维坐标
# 比如 x、y 都从 0 到 1，分别取 11 个点
x = np.linspace(0.0, 1.0, 3)   # 形状 (11,)
y = np.linspace(0.0, -1.0, 3)   # 形状 (11,)

# 2. 构造二维网格
# X, Y 都是形状 (11, 11)
X, Y = np.meshgrid(x, y, indexing='xy')

# 3. 在网格上定义一个物理量，例如 u(x,y) = sin(pi x) * cos(pi y)
U = np.sin(np.pi * X) * np.cos(np.pi * Y)
# 此时 U.shape == (11, 11)

# 4. 将二维数组拉平成一维
U_flat1 = U.ravel()      # 形状 (121,)
U_flat2 = U.flatten()    # 同样是 (121,)
print(f'U_flat1 = {U_flat1}\nU_flat2 = {U_flat2}')
# 如果需要同时得到对应的坐标，也可以这样做：
# 将 X,Y 拉平，然后堆叠成 (N,2) 的坐标矩阵
XY_flat = np.vstack([X.ravel(), Y.ravel()]).T
# XY_flat.shape == (121, 2)，每行是一个 (x, y) 对应点

# 打印一下确认
print("X shape:", X.shape, "Y shape:", Y.shape, "U shape:", U.shape)
print("Flatten 后 U_flat:", U_flat1.shape)
print("坐标矩阵 XY_flat:", XY_flat.shape)


U_flat1 = [ 0.00000000e+00  1.00000000e+00  1.22464680e-16  0.00000000e+00
  6.12323400e-17  7.49879891e-33 -0.00000000e+00 -1.00000000e+00
 -1.22464680e-16]
U_flat2 = [ 0.00000000e+00  1.00000000e+00  1.22464680e-16  0.00000000e+00
  6.12323400e-17  7.49879891e-33 -0.00000000e+00 -1.00000000e+00
 -1.22464680e-16]
X shape: (3, 3) Y shape: (3, 3) U shape: (3, 3)
Flatten 后 U_flat: (9,)
坐标矩阵 XY_flat: (9, 2)


In [7]:
import numpy as np

# 方法1：直接定义二维数组
physical_2d = np.array([[1, 2], [4, 5], [7, 8]])
# 方法1：flatten() - 返回副本（不影响原数组）
physical_1d_flatten = physical_2d.flatten()

# 方法2：ravel() - 返回视图（修改视图会影响原数组）
physical_1d_ravel = physical_2d.ravel()

# 方法3：reshape(-1) - 显式展平（等效于ravel()）
physical_1d_reshape = physical_2d.reshape(-1)



In [8]:
print("原始二维数组：\n", physical_2d)
print("flatten结果：", physical_1d_flatten)
print("ravel结果：  ", physical_1d_ravel)


原始二维数组：
 [[1 2]
 [4 5]
 [7 8]]
flatten结果： [1 2 4 5 7 8]
ravel结果：   [1 2 4 5 7 8]


In [9]:
import numpy as np

# 假设 num_cell 已知
num_cell = 28
V_leftBC  = -1
V_rightBC =  1
diff      = (V_rightBC - V_leftBC) / num_cell

# 1) 生成 1D 节点上的电势
V1D = np.zeros(num_cell+1)
V1D[0]            = V_leftBC
V1D[1:num_cell]   = V_leftBC + diff * np.arange(1, num_cell)
V1D[num_cell]     = V_rightBC

# 2) 把它沿 y 方向复制成 2D
#    结果是 shape = (num_cell+1, num_cell+1)，
#    每一行（固定 y）都跟 V1D 一样
V2D = np.tile(V1D, (num_cell+1, 1))

# 如果你只需要 cell-centered （即 shape=(num_cell,num_cell)），
# 可以直接去掉最后一行和最后一列：
V_cells = V2D[:-1, :-1]
print(f'V2D.shape = {V2D.shape}\nV_cells.shape = {V_cells.shape}')

V2D.shape = (29, 29)
V_cells.shape = (28, 28)


In [1]:
import numpy as np


def init_voltage_2d(nx, ny, V_leftBC, V_rightBC):
    """
    初始化二维网格上的电压，使其在 x 方向上从 V_leftBC 到 V_rightBC 均匀分布。

    参数:
    - nx: 网格在 x 方向的节点数（列数）。
    - ny: 网格在 y 方向的节点数（行数）。
    - V_leftBC: 左边界电压值。
    - V_rightBC: 右边界电压值。

    返回:
    - V: 一维 numpy 数组，长度为 nx * ny，按行优先（row-major）存储电压分布。
    - V2d: 二维 numpy 数组，形状为 (ny, nx)，可直接按 (行, 列) 访问。
    """
    # 在 x 方向生成均匀分布的电压值
    x_vals = np.linspace(V_leftBC, V_rightBC, nx)

    # 将 x 方向的电压分布复制到每一行，得到 2D 数组
    V2d = np.tile(x_vals, (ny, 1))

    # 如果需要一维扁平化表示：
    V = V2d.ravel()

    return V, V2d


if __name__ == "__main__":
    # 网格大小
    nx, ny = 62, 62

    # 模拟边界条件参数示例（请替换为实际参数）
    # Vbi: built-in potential；const.Vt: 热电压；params.phi_a, params.phi_c: 电极功函数
    Vbi = 1.0         # 示例值
    const_Vt = 0.02585 # 示例：室温下约 25.85 mV
    phi_a = 0.2       # 示例值
    phi_c = 0.3       # 示例值

    # 计算左右边界电压
    V_leftBC = -((Vbi) / (2 * const_Vt) - phi_a / const_Vt)
    V_rightBC = (Vbi) / (2 * const_Vt) - phi_c / const_Vt

    # 初始化电压
    V_flat, V2d = init_voltage_2d(nx, ny, V_leftBC, V_rightBC)

    # 检查结果
    print(f"V_flat shape: {V_flat.shape}")   # 应为 (3844,)
    print(f"V2d shape: {V2d.shape}")         # 应为 (62, 62)
    
    # 访问示例：按 (i,j) 获取第 i 行、第 j 列的电压
    i, j = 10, 20
    V_ij = V2d[i, j]
    print(f"V[{i},{j}] = {V_ij}")

    # 如果需要按一维索引访问，可以：
    idx = i * nx + j
    print(f"V_flat[{idx}] = {V_flat[idx]}")


V_flat shape: (3844,)
V2d shape: (62, 62)
V[10,20] = -5.263658559786916
V_flat[640] = -5.263658559786916
